In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.cluster import KMeans
from datetime import datetime
from tqdm import tqdm
from Model import ForexTransformer

In [2]:
FILE_PATH = "Dataset\EURUSD\EURUSD_M30_with_features.csv"

<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Stephen\AppData\Local\Temp\ipykernel_14996\891419644.py:1: SyntaxWarning: invalid escape sequence '\E'
  FILE_PATH = "Dataset\EURUSD\EURUSD_M30_with_features.csv"


In [3]:
# Load the dataset with features
data = pd.read_csv(FILE_PATH)

# Ensure DateTime is parsed correctly
data['DateTime'] = pd.to_datetime(data['DateTime'])

In [4]:
# Select features for training (e.g., SMA100, RSI14, Close)
features = ['SMA100', 'RSI14', 'Close']
X = data[features].values

# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to('cuda')

In [5]:
# Predict target price and stop loss for buy/sell conditions
def calculate_target_stop_prices(close_price, condition):
    if condition == 0:  # Buy
        target_price = close_price * 1.02  # 2% increase
        stop_loss_price = close_price * 0.98  # 2% decrease
    elif condition == 1:  # Sell
        target_price = close_price * 0.98  # 2% decrease
        stop_loss_price = close_price * 1.02  # 2% increase
    else:  # Hold or Nothing
        target_price = stop_loss_price = close_price
    return target_price, stop_loss_price


In [6]:
# Parameters
seq_length = 14  # Number of time steps
num_heads = 4
num_layers = 2
output_dim = 4  # Buy, Sell, Hold, Nothing

In [7]:
# Prepare sequential data
X_seq = []
for i in range(len(X_scaled) - seq_length + 1):
    X_seq.append(X_scaled[i:i + seq_length])
X_seq = torch.tensor(X_seq, dtype=torch.float32).to('cuda')

C:\Users\Stephen\AppData\Local\Temp\ipykernel_14996\74650612.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X_seq = torch.tensor(X_seq, dtype=torch.float32).to('cuda')


In [8]:
# Generate pseudo-labels (unsupervised clustering approach)
def generate_pseudo_labels(X_scaled, n_clusters=4):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    pseudo_labels = kmeans.fit_predict(X_scaled)
    pseudo_labels = pseudo_labels[:len(X_scaled) - seq_length + 1]
    return torch.tensor(pseudo_labels, dtype=torch.long).to('cuda')

pseudo_labels = generate_pseudo_labels(X_scaled)

In [9]:
class ForexTransformer(nn.Module):
    def __init__(self, input_dim, seq_length, num_heads, num_layers, output_dim):
        super(ForexTransformer, self).__init__()
        self.embed_dim = 128  # Ensure this is divisible by num_heads
        self.embedding = nn.Linear(input_dim, self.embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.embed_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(0.3)  # Added dropout to prevent overfitting
        self.fc = nn.Linear(self.embed_dim * seq_length, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = self.dropout(x)  # Applied dropout after the transformer layer
        x = x.flatten(start_dim=1)  # Flatten for the fully connected layer
        x = self.fc(x)
        return x

In [10]:
# Initialize model, loss, and optimizer
model = ForexTransformer(input_dim=X_tensor.shape[1], seq_length=seq_length,  num_heads=num_heads, num_layers=num_layers, output_dim=output_dim).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

c:\Program Files\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# MODEL_PATH = "D:/Programing/AI Trader/Model/transformerModel+dropout_seqlength 14_Loss 0.7001_at 20241215-171504.model"
# model.load_state_dict(torch.load(MODEL_PATH))
# model.eval()
# print(f"\nModel: {MODEL_PATH.split('/')[-1]} is loaded.\n")

C:\Users\Stephen\AppData\Local\Temp\ipykernel_14996\4171171468.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


FileNotFoundError: [Errno 2] No such file or directory: 'D:/Programing/AI Trader/Model/transformerModel+dropout_seqlength 14_Loss 0.7001_at 20241215-171504.model'

In [ ]:
# Training loop
def train_transformer(model, X_seq, pseudo_labels, epochs=100, batch_size=4096):
    dataset = torch.utils.data.TensorDataset(X_seq, pseudo_labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for epoch in tqdm(range(epochs), desc="Training Epochs"):
        model.train()
        epoch_loss = 0
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f} ; {timestamp}")
        MODEL_SAVEPATH = f"./Model/transformerModel+dropout_seqlength {seq_length}_Loss {epoch_loss:.4f}_at {timestamp}.model"
        
        # print(MODEL_SAVEPATH)
        
        torch.save(model.state_dict(), MODEL_SAVEPATH)

train_transformer(model, X_seq, pseudo_labels)

Training Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
# timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

# MODEL_SAVEPATH = f"./Model/transformerModel_seqlength {seq_length}_at {timestamp}.pth"

# print(MODEL_SAVEPATH)

# torch.save(model.state_dict(), MODEL_SAVEPATH)
# print(f"Model saved to {MODEL_SAVEPATH}")


In [ ]:
# Example usage
example_data = X_seq[:10]
model.eval()
with torch.no_grad():
    predictions = model(example_data)
conditions = torch.argmax(predictions, axis=1).cpu().numpy()
for i, condition in enumerate(conditions):
    close_price = data.iloc[i]['Close']
    target, stop_loss = calculate_target_stop_prices(close_price, condition)
    print(f"Condition: {condition}, Target Price: {target:.4f}, Stop Loss Price: {stop_loss:.4f}")
